In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from py2neo import Graph

%matplotlib inline

In [2]:
groups = pd.read_csv('meetup/groups.csv')

In [7]:
groups.columns

Index(['group_id', 'category_id', 'category.name', 'category.shortname',
       'city_id', 'city', 'country', 'created', 'description',
       'group_photo.base_url', 'group_photo.highres_link',
       'group_photo.photo_id', 'group_photo.photo_link',
       'group_photo.thumb_link', 'group_photo.type', 'join_mode', 'lat',
       'link', 'lon', 'members', 'group_name', 'organizer.member_id',
       'organizer.name', 'organizer.photo.base_url',
       'organizer.photo.highres_link', 'organizer.photo.photo_id',
       'organizer.photo.photo_link', 'organizer.photo.thumb_link',
       'organizer.photo.type', 'rating', 'state', 'timezone', 'urlname',
       'utc_offset', 'visibility', 'who'],
      dtype='object')

In [13]:
groups['group_photo.photo_link'].iloc[2]

'https://secure.meetupstatic.com/photos/event/5/7/5/8/600_431422360.jpeg'

In [14]:
import requests
pic = requests.get('https://secure.meetupstatic.com/photos/event/5/7/5/8/600_431422360.jpeg')

In [15]:
pic.content

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xe2\x00\x00\x02X\x08\x02\x00\x00\x0012rz\x00\x00\x00\tpHYs\x00\x00\x16%\x00\x00\x16%\x01IR$\xf0\x00\x00 \x00IDATx\x9c,\xbaI\xaf\xe7\xf9u\xdew\xce\xf9\x8e\xbf\xf1?\xdcy\xa8[U\xdd\xcd\xe6,\x8a\xa2(*\x00-#N\x16^\x04\x08\x10@\xeb\xacb @6\x01\x02o\xb2\xcb\xc2\xfb\xbc\x86l\x82\x00A\x14!\xb1c8\x86\x820\xb4l\x91\xe2\xd4lvWuWW\xdd\xba\xf3\xbd\xff\xf97~\xe7,Zo\xe2s\x9e\xf3y\x1e\xfe/\xfe\x87\xff\xe9\xee\xee\xe1\xf2\xabK\x8e\xfc\x87\x7f\xf2\xa3?\xfb\x8f~ZN\xe6\\+\xa5\xc4_\xfdo\xff\xf3\xbf\xfe\x97\x7fE\t.\xce\xce\x82\x0bm\xd3s)\x03\xb0\xd1x\x8c)\xcb\xb2\x04\xf1{?\xf8\x93\xff\xf2\x9f\xfd\xe7\xdf\xfax\xb2\x1e\xe3_\xfd\xdf\xff\xd7\xcf~\xf67\xdf<\xfd\xf0\xdb\x1f~\xbb\xd9m\xff\xee\xe7\x7f\xf7\xea7\xaf\x9c\xf7:\xcf\xb3\xa2(\xca\x8a\x93t\xd6\xc5\x98\x94RJ\x89\x10\xc28\x8e\xde{\xce\x99*B\x08\xb6i\xda\xa6i\xbd1\x8a3!%#p>\xf4\x9d\xb0\xde\x07oB\xf0\xc4\x03\x17\xc8\x18\x00\x86\x10\x82\xf3#$@d\x8c\x84RyU\xd5RfC?6m3\x8e\x9d\x10\x1c\x11\x87a\x00\x04)\xa5\xb3n4#\x0

In [3]:
categories = pd.read_csv('meetup/categories.csv')
cities = pd.read_csv('meetup/cities.csv')
events = pd.read_csv('meetup/events.csv')
members = pd.read_csv('meetup/members.csv',encoding='cp1252')
topics = pd.read_csv('meetup/topics.csv', encoding='cp1252')
venues = pd.read_csv('meetup/venues.csv', encoding='cp1252')
group_topics = pd.read_csv('meetup/groups_topics.csv', encoding='cp1252')
members_topics = pd.read_csv('meetup/members_topics.csv', encoding='cp1252')


In [4]:
group_topics.head(3)

,topic_id,topic_key,topic_name,group_id
0,83,sportsfans,Sports Fan,241031
1,83,sportsfans,Sports Fan,289172
2,83,sportsfans,Sports Fan,295444


In [5]:
topics.head(3)

,topic_id,description,link,members,topic_name,urlkey,main_topic_id
0,83,Meet with others in your local area who are Sp...,http://sportsfans.meetup.com/,471594,Sports Fan,sportsfans,10451
1,130,Meet with Latin Music fans in your town.,http://latinmusic.meetup.com/,759757,Latin Music,latinmusic,15018
2,182,Want to practice your English? Meetup with oth...,http://esl.meetup.com/,3176752,English as a Second Language,esl,10454


In [6]:
members_topics.head(3)

,topic_id,topic_key,topic_name,member_id
0,83,Sports Fan,sportsfans,121483
1,83,Sports Fan,sportsfans,165644
2,83,Sports Fan,sportsfans,327482


In [7]:
members_topics.shape

(3195245, 4)

In [8]:
events.head(3)

,event_id,created,description,duration,event_url,fee.accepts,fee.amount,fee.currency,fee.description,fee.label,...,venue.lon,venue.name,venue.phone,venue.repinned,venue.state,venue.zip,visibility,waitlist_count,why,yes_rsvp_count
0,153868222,2013-12-03 21:24:29,Solve the murder case while eating a 3 course ...,20700,https://www.meetup.com/SanFranciscoStartupFun/...,others,0.0,not_found,per person,Price,...,-122.405690,Little Szechuan,-1,1,CA,-1,public,0,not_found,72
1,184167702,2014-05-20 18:52:00,NOTE: This event is EVERY FRIDAY!! Signup is a...,77400,https://www.meetup.com/hr-and-tech-sf/events/1...,others,0.0,not_found,per person,Price,...,-122.405457,Horizon Lounge,-1,0,CA,-1,public,0,not_found,179
2,215200502,2014-10-23 16:18:44,We'll be heading out with a group of travelers...,37800,https://www.meetup.com/hr-and-tech-sf/events/2...,others,0.0,not_found,per person,Price,...,-122.407394,taste of vietnam noodle bar,-1,0,CA,-1,public,0,not_found,45


In [9]:
events.columns

Index(['event_id', 'created', 'description', 'duration', 'event_url',
       'fee.accepts', 'fee.amount', 'fee.currency', 'fee.description',
       'fee.label', 'fee.required', 'group.created', 'group.group_lat',
       'group.group_lon', 'group_id', 'group.join_mode', 'group.name',
       'group.urlname', 'group.who', 'headcount', 'how_to_find_us',
       'maybe_rsvp_count', 'event_name', 'photo_url', 'rating.average',
       'rating.count', 'rsvp_limit', 'event_status', 'event_time', 'updated',
       'utc_offset', 'venue.address_1', 'venue.address_2', 'venue.city',
       'venue.country', 'venue_id', 'venue.lat',
       'venue.localized_country_name', 'venue.lon', 'venue.name',
       'venue.phone', 'venue.repinned', 'venue.state', 'venue.zip',
       'visibility', 'waitlist_count', 'why', 'yes_rsvp_count'],
      dtype='object')

In [10]:
members.head(3)

,member_id,bio,city,country,hometown,joined,lat,link,lon,member_name,state,member_status,visited,group_id
0,3,not_found,New York,us,"New York, NY",2007-05-01 22:04:37,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2009-09-18 18:32:23,490552
1,3,not_found,New York,us,"New York, NY",2011-01-23 14:13:17,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-03-20 01:02:11,1474611
2,3,"Hi, I'm Matt. I'm an entrepreneur who has star...",New York,us,"New York, NY",2010-12-30 18:47:34,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-01-18 20:37:23,1490492


In [11]:
cities.head(3)

,city,city_id,country,distance,latitude,localized_country_name,longitude,member_count,ranking,state,zip
0,West New York,7093,us,2524.541,40.790001,USA,-74.010002,661,32,NJ,7093
1,New York,10001,us,2526.837,40.750000,USA,-73.989998,229371,0,NY,10001
2,New York Mills,13417,us,2392.162,43.099998,USA,-75.290001,22,109,NY,13417


In [12]:
categories.head(3)

,category_id,category_name,shortname,sort_name
0,1,Arts & Culture,Arts,Arts & Culture
1,2,Career & Business,Business,Career & Business
2,3,Cars & Motorcycles,Auto,Cars & Motorcycles


In [13]:
venues.head(3)

,venue_id,address_1,city,country,distance,lat,localized_country_name,lon,venue_name,rating,rating_count,state,zip,normalised_rating
0,5286,424 Park Ave S,New York,us,0.0,40.744259,USA,-73.983749,Starbucks Coffee,4.00,51.0,NY,10016,3.92
1,5293,Union Square,New York,us,0.0,40.731390,USA,-73.988400,Virgin Megastore (cafe),2.83,109.0,NY,10003,2.80
2,8356,141 West 72nd St.,New York,us,0.0,40.778275,USA,-73.980095,Krispy Kreme Doughnuts,2.11,18.0,NY,10028,2.00


In [14]:
venues.isnull().sum()

venue_id                   0
address_1                 34
city                       0
country                    0
distance                   0
lat                        0
localized_country_name     0
lon                        0
venue_name                21
rating                     0
rating_count               0
state                      0
zip                        0
normalised_rating          0
dtype: int64

In [15]:
# Create relationship between events and group
'''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///groups.csv' as group
MATCH (e: Events), (g: Groups)
WHERE e.event_id = event.event_id and g.group_id = event.group_id
MERGE (e)-[r:CREATED_BY]->(g)'''

#create relationship between groups and categories
'''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///groups.csv' as group
MATCH (g: Groups), (cat: Categories)
WHERE g.group_id = group.group_id and cat.category_id = group.category_id
MERGE (g)-[r:HAS_CATEGORY]->(cat)'''

# Create relationship between members and groups
'''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///members.csv' as member
MATCH (m: Members), (g: Groups)
WHERE g.group_id = member.group_id and m.member_id = member.member_id
MERGE (m)-[r:HAS_GROUP]->(g)'''

# Create relationship between events and venues
'''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///events.csv' as event
MATCH (e: Events), (v: Venues)
WHERE e.event_id = event.event_id and v.venue_id = event.venue_id
MERGE (e)-[r:HAS_VENUE]->(v)'''

# Create relationship between groups and topics
'''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///groups_topics.csv' as gt
MATCH (g: Groups), (t: Topics)
WHERE g.group_id = gt.group_id and t.topic_id = gt.topic_id
MERGE (g)-[r:HAS_TOPIC]->(t)'''

# Create relationship between members and topics
'''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///members_topics.csv' as mt
MATCH (m: Members), (t: Topics)
WHERE m.member_id = mt.member_id and t.topic_id = mt.topic_id
MERGE (m)-[r:LIKES_TOPIC]->(t)'''

# Create relationship between groups and cities
'''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///groups.csv' as group
MATCH (g: Groups), (c: Cities)
WHERE g.group_id = group.group_id and c.city_id = group.city_id
MERGE (g)-[r:HAS_CITY]->(c)'''



"USING PERIODIC COMMIT\nLOAD CSV WITH HEADERS FROM 'file:///groups.csv' as group\nMATCH (g: Groups), (c: Cities)\nWHERE g.group_id = group.group_id and c.city_id = group.city_id\nMERGE (g)-[r:HAS_CITY]->(c)"

In [3]:
graph = Graph('meetup_db/installation-3.4.6/import', username = 'neo4j', password = 'Evangeline123')

In [42]:
# tx = graph.begin()
statement = '''CALL algo.pageRank.stream(
'MATCH (t:Topics) WHERE exists( (t)-[:LIKES_TOPIC]-() ) RETURN id(t) as id',
'MATCH (t:Topics)-[:LIKES_TOPIC]-(m:Members) RETURN id(m) as source, id(t) as target'
) YIELD node,score with node,score order by score desc limit 20
RETURN node {.name}, score'''
data = graph.run(statement).data()

In [43]:
data

[{'node': {'name': None}, 'score': 13248.741945999998},
 {'node': {'name': 'Tech'}, 'score': 8552.601154},
 {'node': {'name': None}, 'score': 6747.7779215},
 {'node': {'name': None}, 'score': 5259.1871249999995},
 {'node': {'name': 'New Technology'}, 'score': 4553.000341999999},
 {'node': {'name': 'Social'}, 'score': 3663.923802},
 {'node': {'name': 'Outdoors'}, 'score': 3345.0990444999998},
 {'node': {'name': 'Career & Business'}, 'score': 3272.438316},
 {'node': {'name': 'Social Networking'}, 'score': 3074.0718524999997},
 {'node': {'name': 'Web Technology'}, 'score': 2529.6816685},
 {'node': {'name': 'Professional Networking'}, 'score': 2478.1267629999998},
 {'node': {'name': 'Technology'}, 'score': 2386.3229465},
 {'node': {'name': 'Art'}, 'score': 2245.657271},
 {'node': {'name': 'Mobile Technology'}, 'score': 1770.685567},
 {'node': {'name': 'Wine'}, 'score': 1763.9897515000002},
 {'node': {'name': 'Sports and Recreation'}, 'score': 1715.4517935},
 {'node': {'name': 'Dancing'}, '

In [51]:
graph.schema.relationship_types

frozenset({'CREATED_BY',
           'HAS_CATEGORY',
           'HAS_CITY',
           'HAS_GROUP',
           'HAS_TOPIC',
           'HAS_VENUE',
           'LIKES_TOPIC'})

In [61]:
statement1 = '''CALL algo.pageRank.stream('Groups', 'HAS_GROUP', {iterations:20, dampingFactor:0.85})
YIELD nodeId, score

MATCH (node) WHERE id(node) = nodeId

RETURN node.name AS group,score
ORDER BY score DESC LIMIT 20'''
data2 = graph.run(statement1).data()

KeyboardInterrupt: 

In [53]:
data2

[{'node': {'name': None}, 'score': 13248.741945999998},
 {'node': {'name': 'Tech'}, 'score': 8552.601154},
 {'node': {'name': None}, 'score': 6747.7779215},
 {'node': {'name': None}, 'score': 5259.1871249999995},
 {'node': {'name': 'New Technology'}, 'score': 4553.000341999999},
 {'node': {'name': 'Social'}, 'score': 3663.923802},
 {'node': {'name': 'Outdoors'}, 'score': 3345.0990444999998},
 {'node': {'name': 'Career & Business'}, 'score': 3272.438316},
 {'node': {'name': 'Social Networking'}, 'score': 3074.0718524999997},
 {'node': {'name': 'Web Technology'}, 'score': 2529.6816685},
 {'node': {'name': 'Professional Networking'}, 'score': 2478.1267629999998},
 {'node': {'name': 'Technology'}, 'score': 2386.3229465},
 {'node': {'name': 'Art'}, 'score': 2245.657271},
 {'node': {'name': 'Mobile Technology'}, 'score': 1770.685567},
 {'node': {'name': 'Wine'}, 'score': 1763.9897515000002},
 {'node': {'name': 'Sports and Recreation'}, 'score': 1715.4517935},
 {'node': {'name': 'Dancing'}, '

In [54]:
members.head()

,member_id,bio,city,country,hometown,joined,lat,link,lon,member_name,state,member_status,visited,group_id
0,3,not_found,New York,us,"New York, NY",2007-05-01 22:04:37,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2009-09-18 18:32:23,490552
1,3,not_found,New York,us,"New York, NY",2011-01-23 14:13:17,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-03-20 01:02:11,1474611
2,3,"Hi, I'm Matt. I'm an entrepreneur who has star...",New York,us,"New York, NY",2010-12-30 18:47:34,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-01-18 20:37:23,1490492
3,3,"Hi, I'm Matt. I'm an entrepreneur who has star...",New York,us,"New York, NY",2011-01-03 14:45:21,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-07-23 03:42:28,1515830
4,3,"Hi, I'm Matt. I'm an entrepreneur who has star...",New York,us,"New York, NY",2010-12-30 18:34:50,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-06-13 18:33:23,1574965


In [55]:
events.head(3)

,event_id,created,description,duration,event_url,fee.accepts,fee.amount,fee.currency,fee.description,fee.label,...,venue.lon,venue.name,venue.phone,venue.repinned,venue.state,venue.zip,visibility,waitlist_count,why,yes_rsvp_count
0,153868222,2013-12-03 21:24:29,Solve the murder case while eating a 3 course ...,20700,https://www.meetup.com/SanFranciscoStartupFun/...,others,0.0,not_found,per person,Price,...,-122.405690,Little Szechuan,-1,1,CA,-1,public,0,not_found,72
1,184167702,2014-05-20 18:52:00,NOTE: This event is EVERY FRIDAY!! Signup is a...,77400,https://www.meetup.com/hr-and-tech-sf/events/1...,others,0.0,not_found,per person,Price,...,-122.405457,Horizon Lounge,-1,0,CA,-1,public,0,not_found,179
2,215200502,2014-10-23 16:18:44,We'll be heading out with a group of travelers...,37800,https://www.meetup.com/hr-and-tech-sf/events/2...,others,0.0,not_found,per person,Price,...,-122.407394,taste of vietnam noodle bar,-1,0,CA,-1,public,0,not_found,45


In [57]:
events.columns

Index(['event_id', 'created', 'description', 'duration', 'event_url',
       'fee.accepts', 'fee.amount', 'fee.currency', 'fee.description',
       'fee.label', 'fee.required', 'group.created', 'group.group_lat',
       'group.group_lon', 'group_id', 'group.join_mode', 'group.name',
       'group.urlname', 'group.who', 'headcount', 'how_to_find_us',
       'maybe_rsvp_count', 'event_name', 'photo_url', 'rating.average',
       'rating.count', 'rsvp_limit', 'event_status', 'event_time', 'updated',
       'utc_offset', 'venue.address_1', 'venue.address_2', 'venue.city',
       'venue.country', 'venue_id', 'venue.lat',
       'venue.localized_country_name', 'venue.lon', 'venue.name',
       'venue.phone', 'venue.repinned', 'venue.state', 'venue.zip',
       'visibility', 'waitlist_count', 'why', 'yes_rsvp_count'],
      dtype='object')

In [58]:
members.shape[0]

5893886

In [59]:
members.head(3)

,member_id,bio,city,country,hometown,joined,lat,link,lon,member_name,state,member_status,visited,group_id
0,3,not_found,New York,us,"New York, NY",2007-05-01 22:04:37,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2009-09-18 18:32:23,490552
1,3,not_found,New York,us,"New York, NY",2011-01-23 14:13:17,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-03-20 01:02:11,1474611
2,3,"Hi, I'm Matt. I'm an entrepreneur who has star...",New York,us,"New York, NY",2010-12-30 18:47:34,40.72,http://www.meetup.com/members/3,-74.0,Matt Meeker,NY,active,2011-01-18 20:37:23,1490492


In [60]:
events.head(3)

,event_id,created,description,duration,event_url,fee.accepts,fee.amount,fee.currency,fee.description,fee.label,...,venue.lon,venue.name,venue.phone,venue.repinned,venue.state,venue.zip,visibility,waitlist_count,why,yes_rsvp_count
0,153868222,2013-12-03 21:24:29,Solve the murder case while eating a 3 course ...,20700,https://www.meetup.com/SanFranciscoStartupFun/...,others,0.0,not_found,per person,Price,...,-122.405690,Little Szechuan,-1,1,CA,-1,public,0,not_found,72
1,184167702,2014-05-20 18:52:00,NOTE: This event is EVERY FRIDAY!! Signup is a...,77400,https://www.meetup.com/hr-and-tech-sf/events/1...,others,0.0,not_found,per person,Price,...,-122.405457,Horizon Lounge,-1,0,CA,-1,public,0,not_found,179
2,215200502,2014-10-23 16:18:44,We'll be heading out with a group of travelers...,37800,https://www.meetup.com/hr-and-tech-sf/events/2...,others,0.0,not_found,per person,Price,...,-122.407394,taste of vietnam noodle bar,-1,0,CA,-1,public,0,not_found,45
